In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import tarfile
import re

from functools import partial
from concurrent.futures import ProcessPoolExecutor
from glob import glob, iglob
from pathlib import Path
from tqdm import tqdm
from unittest.mock import MagicMock

import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from projections.shapefiles import iter_records
from projections import raster, utils


pd.set_option('max_columns', None)

/sd2/Moncho/zonalstats_earth_engine/.venv/lib/python3.8/site-packages/geopandas/_compat.py:106: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.9.1-CAPI-1.14.2). Conversions between both will be slow.
  warnings.warn(


In [2]:
def load_tifs(tars):
    for tar in tars:
        names = []

        with tarfile.open(tar, 'r') as tf:
            # Find gz in tar
            names = [x for x in tf.getnames() if x.endswith('.gz') and 'stable_lights' in x]
            if not names:
                print(f'No stable lights in {tar}')
                continue

            for name in names:
                tf.extract(name, read_path)

        for name in names:
            # Extract tif
            name_path = read_path / name
            os.system(f'gunzip {name_path}')

            tifs = glob(str(read_path / '*.tif'))

            for tif in tifs:
                # Yield path to tif file
                yield tif

            # Clean up
            for file in tifs + [name_path]:
                try:
                    os.remove(file)
                except FileNotFoundError:
                    continue
                    
                    
def save_location_mapping(row_path_tif):
    row, path, tif = row_path_tif
    IMAGE = utils.read_tif(tif)
    shape = row['geometry']
    
    subset = raster.find_subset_with_intersection_area(IMAGE, shape)

    if subset is None:
        with open(path, 'w') as f:
            f.write('')
        return

    subset['adm0'] = row['GID_0']
    subset['adm1'] = row['GID_1']
    subset['adm2'] = row['GID_2']
    
    subset.to_csv(path, index=False)

In [3]:
read_path = Path('../Data/Nightlights/')
output_path = Path('../Output/Nighlights/')
partial_path = utils.make_path(output_path / 'partial_locs')
countries_path = utils.make_path(output_path / 'countries')
ethnic_path = utils.make_path(output_path / 'ethnic')
ethnic_countries_path = utils.make_path(output_path / 'ethnic_countries')

tars = sorted(glob(str(read_path / '*.tar')))

In [4]:
geo_df = gpd.read_file('../Shapefiles/preprocessed/all_countries.shp')
geo_df['GID_1'].fillna(geo_df['GID_0'], inplace=True)
geo_df['GID_2'].fillna(geo_df['GID_1'], inplace=True)

# Map raster to polygons

In [5]:
def get_filename_from_row(row, path):
    name = f'{row["GID_2"]}.csv'
    return path / name


def get_filename_with_portion_from_row(row, path):
    portion = f"_p{int(row['portion']):03d}" if row['portion'] else ''
    name = f'{row["GID_2"]}{portion}.csv'
    return path / name


def record_exists(row, path):
    return (
        get_filename_from_row(row, path).exists() or 
        get_filename_with_portion_from_row(row, path).exists()
    )


def yield_missing_records(records, save_path, tif=None):
    for _, row in records.iterrows():
        if record_exists(row, save_path):
            continue
            
        yield row, get_filename_with_portion_from_row(row, save_path), tif

In [6]:
# Only need to map the locations once
# Still loop for the clean up
n_processes = 20
for tif in load_tifs(tars[:1]):
    processing_function = partial(save_location_mapping)
    country_iterator = partial(yield_missing_records, save_path=partial_path, tif=tif)
    
    if n_processes == 1:
        for blob in tqdm(country_iterator(geo_df)):
            processing_function(blob)
    else:
        with ProcessPoolExecutor(n_processes) as tpe:
            for _ in tqdm(tpe.map(processing_function, country_iterator(geo_df)), 
                          total=geo_df.shape[0]):
                pass

gzip: ../Data/Nightlights/F101992.v4b_web.stable_lights.avg_vis.tif already exists;	not overwritten
  0%|▏                                     | 503/117719 [00:09<37:37, 51.93it/s]


In [51]:
gid_with_portions = geo_df.loc[geo_df['portion'].notnull(), 'GID_2'].unique()
for gid in tqdm(gid_with_portions):
    files = list(partial_path.glob(f'{gid}_p*.csv'))
    if not files:
        continue
        
    portions = [utils.read_csv(file) for file in files]
    portions = [x for x in portions if not x.empty]
    
    if not portions:
        continue
    elif len(portions) == 1:
        country = portions[0]
    else:
        county = portions[0].append(portions[1:], ignore_index=True)
        
    county = county.groupby(['lat', 'lon']).sum().reset_index()
    for i in range(3):
        county[f'adm{i}'] = portions[0].iloc[0][f'adm{i}']
    county.to_csv(partial_path / f'{gid}.csv', index=False)

100%|█████████████████████████████████████████| 414/414 [11:53<00:00,  1.72s/it]


# Aggregate

In [5]:
def is_portion(name):
    match = re.match('.*_p\d\d\d.csv$', name)
    return match is not None


def get_files_by_country(path):
    files_by_country = {}
    for file in path.glob('*.csv'):
        if is_portion(file.name):
            continue
            
        file = Path(file)
        country = file.name[:3]
        if country in files_by_country:
            files_by_country[country].append(file)
        else:
            files_by_country[country] = [file]
            
    return files_by_country


def get_year_from_tif(tif):
    return int(Path(tif).name[3:7])


def aggregate_and_save(results, tif, save_path, name, groupby):
    year = Path(tif).name[3:7]    
    
    if '{year}' in name:
        name = name.format(year=year)
        
    save_path = save_path / name

    if save_path.exists():
        return False

    # Get raster subset
    mask = results['lon'] > -999
    
    if mask.sum() > 0:
        IMAGE = utils.read_tif(tif)
        increment = raster.get_increment_from_tif(IMAGE)
        
        shape = MagicMock()
        shape.bounds = (
            results.loc[mask, 'lon'].min() - increment,
            results.loc[mask, 'lat'].min() - increment,
            results['lon'].max() + increment,
            results['lat'].max() + increment
        )

        subset = raster.get_df_by_maximum_bounds(IMAGE, shape, geo=False)
        subset['lon'] = np.round(subset['lon'], 6)
        subset['lat'] = np.round(subset['lat'], 6)

        # Combine with results
        if 'value' in results.columns:
            results.drop(columns='value', inplace=True)
        tresults = results.merge(subset, on=['lon', 'lat'], how='left')
        tresults['value'].fillna(0, inplace=True)  # This fills the -999
    else:
        tresults = results.copy()
        tresults['value'] = 0

    # Aggregate
    pivot = tresults.groupby(groupby)[['intersection_area', 'value']].sum()
    pivot['year'] = int(year)

    # Save
    pivot.reset_index().to_csv(save_path, index=False)
    
    return True


def load_nl_files(files):
    results = []
    for file in files:
        try:
            df = pd.read_csv(file)
        except pd.errors.EmptyDataError:
            continue
            
        if df.empty or 'intersection_area' not in df.columns:
            continue
            
        try:
            results.append(df[pd.to_numeric(df['intersection_area']) > 0])    
        except Exception as e:
            print(df.columns)
            print(file)
            raise e
        
    results = results[0].append(results[1:], ignore_index=True)
    results['lon'] = np.round(results['lon'], 6)
    results['lat'] = np.round(results['lat'], 6)
    
    return results


def iter_files_by_country(files_by_country, tif, path):
    for country, files in files_by_country.items():
        year = get_year_from_tif(tif)
        name = f'{country}_{year}.csv'
        if (path / name).exists():
            continue
            
        yield country, files, name, tif

In [6]:
def load_aggregate_and_save(blob):
    country, files, name, tif_name = blob    
    
    # Load polygons
    results = load_nl_files(files)
    results['adm1'].fillna('', inplace=True)
    results['adm2'].fillna('', inplace=True)
    results.drop_duplicates(inplace=True)

    aggregate_and_save(results, tif, countries_path, name, groupby=['adm0', 'adm1', 'adm2'])

    
n_processes = 5
files_by_country = get_files_by_country(partial_path)

for tif in load_tifs(tars):
    with ProcessPoolExecutor(n_processes) as tpe:
        for _ in tqdm(tpe.map(load_aggregate_and_save, 
                              iter_files_by_country(files_by_country, tif, countries_path)), 
                      total=len(files_by_country), desc=tif):
            pass

../Data/Nightlights/F101992.v4b_web.stable_lights.avg_vis.tif: 100%|█| 256/256 [
../Data/Nightlights/F101993.v4b_web.stable_lights.avg_vis.tif: 100%|█| 256/256 [
../Data/Nightlights/F121994.v4b_web.stable_lights.avg_vis.tif: 100%|█| 256/256 [
../Data/Nightlights/F121995.v4b_web.stable_lights.avg_vis.tif: 100%|█| 256/256 [
../Data/Nightlights/F121996.v4b_web.stable_lights.avg_vis.tif: 100%|█| 256/256 [
../Data/Nightlights/F141997.v4b_web.stable_lights.avg_vis.tif: 100%|█| 256/256 [
../Data/Nightlights/F141998.v4b_web.stable_lights.avg_vis.tif: 100%|█| 256/256 [
../Data/Nightlights/F141999.v4b_web.stable_lights.avg_vis.tif: 100%|█| 256/256 [
../Data/Nightlights/F152000.v4b_web.stable_lights.avg_vis.tif: 100%|█| 256/256 [
../Data/Nightlights/F152001.v4b_web.stable_lights.avg_vis.tif: 100%|█| 256/256 [
../Data/Nightlights/F152002.v4b_web.stable_lights.avg_vis.tif: 100%|█| 256/256 [
../Data/Nightlights/F152003.v4b_web.stable_lights.avg_vis.tif: 100%|█| 256/256 [
../Data/Nightlights/F162004.

# Combine aggregations

In [7]:
results = []
for file in tqdm(countries_path.glob('*.csv')):
    df = utils.read_csv(file)
    if not df.empty:
        results.append(df)
        
results = results[0].append(results[1:], ignore_index=True)
print(results.shape)
results.to_csv(output_path / 'nightlights.csv', index=False)

5632it [00:12, 448.60it/s]


(1027994, 6)


In [12]:
df = pd.read_csv(output_path / 'nightlights.csv')
df.head()

,adm0,adm1,adm2,intersection_area,value,year
0,ALB,ALB.10_1,ALB.10.1_1,1.147588e+09,70.0,1992
1,ALB,ALB.10_1,ALB.10.2_1,1.056770e+09,0.0,1992
2,ALB,ALB.10_1,ALB.10.3_1,1.301028e+09,594.0,1992
3,ALB,ALB.11_1,ALB.11.1_1,4.093430e+08,290.0,1992
4,ALB,ALB.11_1,ALB.11.2_1,1.227538e+09,3004.0,1992


# Ethnic
# Map raster to Ethnic locs

In [8]:
def iter_records(adm, save_path):
    for _, row in tqdm(adm.iterrows(), total=adm.shape[0]):
        name = f'{row["GID_0"]}_{row["NAME"]}.csv'
        name_path = Path(save_path) / name
            
        yield row, name_path

def iter_and_skip_records(adm, save_path, tif):
    for row, name_path in iter_records(adm, save_path):
        if name_path.exists():
            continue
            
        yield row, name_path, tif   
        

def save_location_mapping(blob):
    row, name_path, tif = blob
    shape = row['geometry']
    
    IMAGE = utils.read_tif(tif)
    subset = raster.find_subset_with_intersection_area(IMAGE, shape)
    if subset is None:
        print('No intersection found for', name_path)
        with open(name_path, 'w') as f:
            f.write('')
        return

    for col in ['NAME', 'TRIBE_CODE', 'GID_0']:
        subset[col] = row[col]
    
    subset.to_csv(name_path, index=False)
    
# Load shapes
adm = gpd.read_file('../Shapefiles/ethnic_preprocessed/tribe_adm0_s.shp')

In [35]:
# Map locations
n_processes = 15
for tif in load_tifs(tars[:1]):    
    i = partial(iter_and_skip_records, save_path=ethnic_path, tif=tif)
    
    if n_processes == 1:
        for blob in i(adm):
            save_location_mapping(blob)
    else:
        with ProcessPoolExecutor(n_processes) as tpe:
            for _ in tqdm(tpe.map(save_location_mapping, i(adm)), total=adm.shape[0]):
                pass

gzip: ../Data/Nightlights/F101992.v4b_web.stable_lights.avg_vis.tif already exists;	not overwritten
  0%|                                         | 12/5053 [00:06<43:11,  1.95it/s]


In [9]:
gid_with_portions = adm.loc[adm['portion'].notnull(), ['GID_0', 'NAME']].agg('_'.join, axis=1).unique()
for gid in tqdm(gid_with_portions):
    files = list(ethnic_path.glob(f'{gid}_p*.csv'))
    if not files:
        continue
        
    portions = [utils.read_csv(file) for file in files]
    portions = [x for x in portions if not x.empty]
    
    if not portions:
        continue
    elif len(portions) == 1:
        country = portions[0]
    else:
        country = portions[0].append(portions[1:], ignore_index=True)
        
    country = country.groupby(['lat', 'lon']).sum().reset_index()
    l
    for i in range(3):
        country[f'adm{i}'] = portions[0].iloc[0][f'adm{i}']
    country.to_csv(partial_path / f'{gid}.csv')

100%|████████████████████████████████████████| 134/134 [00:00<00:00, 915.85it/s]


# Aggregate ethnic

In [34]:
# for file in ethnic_path.glob('*.csv'):
#     f = pd.read_csv(file)
#     if (
#         str(f['lon'].dtype) != 'float64' 
#         or str(f['lat'].dtype) != 'float64'
#         or f['lon'].max() > 50000
#         or f['lat'].max() > 20000
#     ):
#         os.remove(file)
#         print(file)

../Output/Nighlights/ethnic/MRT_BERABISH.csv


In [10]:
def ethnic_load_aggregate_and_save(blob):
    groupby = ['NAME', 'TRIBE_CODE', 'GID_0']
    country, files, name, tif_name = blob    
    
    # Load polygons
    results = load_nl_files(files)
    for col in groupby:
        results[col].fillna('', inplace=True)
    results.drop_duplicates(inplace=True)

    aggregate_and_save(results, tif_name, ethnic_countries_path, name, groupby=groupby)


files_by_country = get_files_by_country(ethnic_path)

n_processes = 10
with ProcessPoolExecutor(n_processes) as tpe:
    for tif in load_tifs(tars):
        for _ in tqdm(tpe.map(ethnic_load_aggregate_and_save, 
                              iter_files_by_country(files_by_country, tif, ethnic_countries_path)), 
                      total=len(files_by_country), desc=tif):
            pass

../Data/Nightlights/F101992.v4b_web.stable_lights.avg_vis.tif: 100%|█| 57/57 [00
../Data/Nightlights/F101993.v4b_web.stable_lights.avg_vis.tif: 100%|█| 57/57 [00
../Data/Nightlights/F121994.v4b_web.stable_lights.avg_vis.tif: 100%|█| 57/57 [00
../Data/Nightlights/F121995.v4b_web.stable_lights.avg_vis.tif: 100%|█| 57/57 [00
../Data/Nightlights/F121996.v4b_web.stable_lights.avg_vis.tif: 100%|█| 57/57 [00
../Data/Nightlights/F141997.v4b_web.stable_lights.avg_vis.tif: 100%|█| 57/57 [00
../Data/Nightlights/F141998.v4b_web.stable_lights.avg_vis.tif: 100%|█| 57/57 [00
../Data/Nightlights/F141999.v4b_web.stable_lights.avg_vis.tif: 100%|█| 57/57 [00
../Data/Nightlights/F152000.v4b_web.stable_lights.avg_vis.tif: 100%|█| 57/57 [00
../Data/Nightlights/F152001.v4b_web.stable_lights.avg_vis.tif: 100%|█| 57/57 [00
../Data/Nightlights/F152002.v4b_web.stable_lights.avg_vis.tif: 100%|█| 57/57 [00
../Data/Nightlights/F152003.v4b_web.stable_lights.avg_vis.tif: 100%|█| 57/57 [00
../Data/Nightlights/F162004.

In [11]:
results = []
for file in tqdm(ethnic_countries_path.glob('*.csv')):
    try:
        results.append(pd.read_csv(file))
    except pd.errors.EmptyDataError:
        continue
results = results[0].append(results[1:], ignore_index=True)
print(results.shape)
results.drop_duplicates(['NAME', 'GID_0', 'year']).to_csv(output_path / 'nightlights_ethnic.csv', index=False)

1254it [00:01, 633.78it/s]


(31064, 6)
